In [2]:
import json
import os
from typing import Any, Callable

import dotenv
from openai import OpenAI
from openai.types.chat.chat_completion import ChatCompletionMessage

# Load environment variables from a .env file if available
if dotenv.load_dotenv(".env"):
    api_key = os.getenv("OPENAI_API_KEY")
else:
    raise ValueError("EnvironmentError: Failed to load `OPENAI_API_KEY`")

if api_key == "sk-no-key-required":
    # Correct base URL for llama.cpp
    base_url = "http://localhost:8080/v1"

# Set up the OpenAI API configuration
client = OpenAI(api_key=api_key, base_url=base_url)

In [3]:
def get_current_weather(location: str, unit: str = "celsius"):
    """
    Get the current weather in a given location.

    Parameters:
    location (str): The city and state, e.g. San Francisco, CA
    unit (str): The unit of temperature, can be either 'celsius' or 'fahrenheit'. Default is 'celsius'.

    Returns:
    str: A string that describes the current weather.
    """

    # This is a mock function, so let's return a mock weather report.
    weather_report = f"The current weather in {location} is 20 degrees {unit}."
    return weather_report

In [4]:
tool_factory = {
    "get_current_weather": get_current_weather,
}

In [5]:
tools = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]

In [6]:
def get_function(
    message: ChatCompletionMessage, tools: dict[str, Callable]
) -> dict[str, Any]:
    # Note: the JSON response may not always be valid; be sure to handle errors
    name = message.function_call.name
    callback = tools[name]
    arguments = json.loads(message.function_call.arguments)
    content = callback(**arguments)
    response = {
        "role": "function",
        "name": name,
        "content": content,
    }
    return response

In [7]:
def initialize_session() -> list[ChatCompletionMessage]:
    system_prompt = {
        "role": "system",
        "content": "Hello! My name is Mistral. I am a helpful assistant.",
    }
    user_input = {
        "role": "user",
        "content": "Hello! My name is Austin. What is your name?",
    }
    messages = [system_prompt, user_input]
    return messages

In [8]:
def add_message(role: str, content: str) -> ChatCompletionMessage:
    message = {"role": role, "content": content}
    messages.append(message)
    return message

In [9]:
def add_completion(
    response: dict[str, Any], messages: list[dict[str, str]]
) -> dict[str, str]:
    message = response.choices[0].message
    messages.append(dict(message))
    return message

In [10]:
def print_completion(messages: list[ChatCompletionMessage]) -> None:
    for message in messages:
        message = dict(message)
        print("keys:", message)
        print("role:", message["role"])
        if message["role"] in ["system", "user", "assistant"]:
            print("content:", message["content"])
            tool_choice = message.get("tool_choice", None)
            if tool_choice:
                print("tool_choice:", tool_choice)
        else:
            print("name:", message["name"])
            print("content:", json.dumps(message["content"]))
        print()

In [11]:
messages = initialize_session()
print_completion(messages)

keys: {'role': 'system', 'content': 'Hello! My name is Mistral. I am a helpful assistant.'}
role: system
content: Hello! My name is Mistral. I am a helpful assistant.

keys: {'role': 'user', 'content': 'Hello! My name is Austin. What is your name?'}
role: user
content: Hello! My name is Austin. What is your name?



In [12]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    temperature=0,
)
print(response)

APIConnectionError: Connection error.

In [68]:
message = add_completion(response, messages)
print_completion(messages)

keys: {'role': 'system', 'content': 'Hello! My name is Mistral. I am a helpful assistant.'}
role: system
content: Hello! My name is Mistral. I am a helpful assistant.

keys: {'role': 'user', 'content': 'Hello! My name is Austin. What is your name?'}
role: user
content: Hello! My name is Austin. What is your name?

keys: {'content': " Hello Austin! It's nice to meet you. I am Mistral, a helpful assistant. How can I assist you today?", 'refusal': None, 'role': 'assistant', 'audio': None, 'function_call': None, 'tool_calls': None}
role: assistant
content:  Hello Austin! It's nice to meet you. I am Mistral, a helpful assistant. How can I assist you today?



In [69]:
message = add_message(
    "user", "What is the weather like today in New York City, New York?"
)
print_completion(messages)

keys: {'role': 'system', 'content': 'Hello! My name is Mistral. I am a helpful assistant.'}
role: system
content: Hello! My name is Mistral. I am a helpful assistant.

keys: {'role': 'user', 'content': 'Hello! My name is Austin. What is your name?'}
role: user
content: Hello! My name is Austin. What is your name?

keys: {'content': " Hello Austin! It's nice to meet you. I am Mistral, a helpful assistant. How can I assist you today?", 'refusal': None, 'role': 'assistant', 'audio': None, 'function_call': None, 'tool_calls': None}
role: assistant
content:  Hello Austin! It's nice to meet you. I am Mistral, a helpful assistant. How can I assist you today?

keys: {'role': 'user', 'content': 'What is the weather like today in New York City, New York?'}
role: user
content: What is the weather like today in New York City, New York?



In [70]:
response = get_chat_completion(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools,
)

print(response)

APIConnectionError: Connection error.

In [ ]:
message = add_completion(response, messages)
print_completion(messages)

In [ ]:
print(message)

In [ ]:
function = get_function(message, function_factory)
print(function)

In [212]:
messages.append(function)

In [ ]:
print_completion(messages)

In [ ]:
response = get_chat_completion(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="auto",  # auto is default, but we'll be explicit
)

print(response)

In [ ]:
add_completion(response, messages)

In [ ]:
print_completion(messages)

In [ ]:
add_message("user", "Thank you! How should I dress for that kind of temperature?")
print_completion(messages)

In [ ]:
# Call the model again with the updated messages
response = get_chat_completion(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="auto",  # auto is default, but we'll be explicit
)

print(response)

In [ ]:
add_completion(response, messages)

In [ ]:
print_completion(messages)